In [11]:
from typing import List, Optional
from enum import Enum
from pydantic import BaseModel, Field, FilePath

In [19]:
# Cubic models
class vdW1(BaseModel):
    a: float = Field(annotation="The parameter a in vdW")
    b: float = Field(annotation="The parameter b in vdW")
        
class vdW(BaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', annotation='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', annotation='The critical pressures in Pa')
        
# Helper types for cubic models
class AlphaOptions(str, Enum):
    Twu = "Twu"
class CubicAlpha(BaseModel):
    type_: AlphaOptions = Field(alias='type', annotation='The type of the alpha function')
    c: List[float] = Field(min_length=3, max_length=3, annotation='The set of coefficients')
        
class PR(BaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', annotation='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', annotation='The critical pressures in Pa')
    acentric: List[float] = Field(annotation='The acentric factors')
    alpha: CubicAlpha = Field(default=None, annotation='The alpha function used in the attractive part')
    kmat: Optional[List[List[float]]] = Field(default=None, annotation="The NxN matrix of kij factors")
        
SRK = PR

In [20]:
class CPAPure(BaseModel):
    a0i: float = Field(alias="a0i / Pa m^6/mol^2")
    bi: float = Field(alias="bi / m^3/mol")
    c1: float = Field(alias="c1")
    Tc: float = Field(alias="Tc / K")
    epsAB: float = Field(alias="epsABi / J/mol")
    betaAB: float = Field(alias="betaABi")
    
class CPACubicEnum(str, Enum):
    PR = "PR"
    SRK = "SRK"
    
class CPA(BaseModel):
    cubic: CPACubicEnum
    pures: List[CPAPure]
    R: float = Field(alias="R_gas / J/mol/K")

In [31]:
class multifluid(BaseModel):
    components: List[FilePath | str]
    root: Optional[str]
    departure: FilePath | List[dict]
    BIP: FilePath | List[dict]
    flags: Optional[object]

In [26]:
# Some model potentials requiring arguments
class SW_EspindolaHeredia2009(BaseModel):
    lambda_: float = Field(alias='lambda', annotation='The well width, where the attractive part goes includes r=(sigma,lambda*sigma)')
    
class EXP6_Kataoka1992(BaseModel):
    alpha: float = Field(annotation='The alpha parameter controlling the hardness of the EXP-6 potential')
        
class Mie_Pohl2023(BaseModel):
    lambda_a: float = Field(annotation='The attractive exponent; the repulsive exponent is 6')
        
class TwoCLJFAuthors(str, Enum):
    Mecke = "2CLJF_Mecke"
    Lisal = "2CLJF_Lisal"
    
class TwoCJLF_Dipole(BaseModel):
    author: TwoCLJFAuthors
    Lstar2: float = Field(annotation="The dimensionless separation of centers")
    mustar2: float = Field(annotation='The dipole moment squared, expressed in simulation units')
        
class TwoCJLF_Quadrupole(BaseModel):
    author: TwoCLJFAuthors
    Lstar2: float = Field(annotation="The dimensionless separation of centers")
    Qstar2: float = Field(annotation='The quadrupole moment squared, expressed in simulation units')

In [30]:
class ECSHuberEly1994ReferenceFluid(BaseModel):
    name: str = Field(annotation="The name of the fluid, could be an absolute path to the .json of a multifluid EOS")
    acentric: float = Field(annotation="The acentric factor")
    Z_crit: float  = Field(annotation="The critical compressibility factor; Z = p/(rho*R*T)")
    T_crit: float = Field(alias="T_crit / K", annotation="The critical temperature")
    rhomolar_crit: float = Field(alias="rhomolar_crit / mol/m^3", annotation="The critical density")
        
class ECSHuberEly1994Fluid(BaseModel):
    acentric: float = Field(annotation="The acentric factor")
    Z_crit: float  = Field(annotation="The critical compressibility factor; Z = p/(rho*R*T)")
    T_crit: float = Field(alias="T_crit / K", annotation="The critical temperature")
    rhomolar_crit: float = Field(alias="rhomolar_crit / mol/m^3", annotation="The critical density")
    f_T_coeffs: List[float] = Field(min_length=2, max_length=2, annotation='The array containing the coefficients alpha_1 and alpha_2 used in f');
    h_T_coeffs: List[float] = Field(min_length=2, max_length=2, annotation='The array containing the coefficients beta_1 and beta_2 used in h');
        
class MultifluidECSHuberEly1994(BaseModel):
    reference_fluid: ECSHuberEly1994ReferenceFluid
    fluid: ECSHuberEly1994Fluid

In [32]:
import json
    
examples = [
    {"kind": "vdW1", "model":{"a": 1.0, "b": 2.0}},
    {"kind": "PR", "model": {"Tcrit / K": [190], "pcrit / Pa": [3.5e6], "acentric": [0.11], "alpha": {"type": "Twu", "c":[1,2,3]}}}
]
for example in examples:
    klass = locals()[example['kind']]
    klass.model_validate(example['model'])
    
# Put each of the schemas into a dictionary where the key is the 
# model kind (as a string) with the value being the schema for this particular 
# model kind
schemas = {}
for klass in [vdW1, vdW, PR, SRK, CPA, 
              SW_EspindolaHeredia2009, EXP6_Kataoka1992, Mie_Pohl2023, 
              (TwoCJLF_Dipole, "2CLJF-Dipole"),
              (TwoCJLF_Quadrupole, "2CLJF-Quadrupole"),
              (MultifluidECSHuberEly1994, "multifluid-ECS-HuberEly1994"),
              MultifluidECSHuberEly1994,
             ]:
    if isinstance(klass, tuple) and len(klass) == 2:
        schemas[klass[1]] = klass[0].model_json_schema()
    else:
        schemas[klass.__name__] = klass.model_json_schema()
    
with open("schemas.json",'w') as fp:
    fp.write(json.dumps(schemas, indent=2))